# Testing a Model
The purpose of this notebook is to quantitatively test trained model using test data to ensure training has occurred correctly. Test data was obtained from [the lambeq docs](https://github.com/CQCL/lambeq-docs/blob/main/docs/examples/datasets/mc_test_data.txt).

In [3]:
from lambeq import TketModel, RemoveCupsRewriter, BobcatParser, AtomicType, IQPAnsatz
from pytket.extensions.qiskit import AerBackend
import numpy as np

First we define an accuracy-measurement function, which computes the accuracy of the model.

In [4]:
def accuracy(labels, results):
    correct_answers = np.sum([np.round(results[i]) == label for i, label in enumerate(labels)])  # Find the number of correct answers
    accuracy = correct_answers/(2*len(labels))  # Factor of 2 because the np.round() causes double-counting
    return accuracy

Next, load the model from the saved checkpoints.

In [5]:
backend = AerBackend()
backend_config = {
    'backend': backend,
    'compilation': backend.default_compilation_pass(2),
    'shots': 1024
}

model = TketModel.from_checkpoint("model.lt", backend_config=backend_config)

Next, we parse the test data and turn them into DisCoCat diagrams.

In [7]:
# Read File
labels, sentences = [], []
with open("data/test_data.txt") as f:
    for line in f:
        t = int(line[0])  # Either 0 or 1 depending on whether about food or IT
        labels.append([t, 1-t])  # Converting to 2D array
        sentences.append(line[1:].strip())

# Create diagrams
parser = BobcatParser()
remove_cups = RemoveCupsRewriter()

test_diagrams = [remove_cups(diagram) for diagram in parser.sentences2diagrams(sentences)]

KeyboardInterrupt: 

Finally, we convert the diagrams to ansatz PQCs, feed our them into the model and evaluate the model's performance using the function defined above.

In [55]:
# Convert ot circuit diagrams
ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 1}, n_layers=1, n_single_qubit_params=3)
test_circuits= [ansatz(diagram) for diagram in test_diagrams]

# Feed into model + evaluate accuracy
acc = accuracy(labels, model(test_circuits))
print(f'Accuracy is {acc}')

Accuracy is 0.8666666666666667
